# Example of semi-supervised gan for mnist

Based on https://machinelearningmastery.com/semi-supervised-generative-adversarial-network/

In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

%matplotlib notebook

Using TensorFlow backend.


In [2]:
# custom activation function
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

In [3]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # image input
    in_image = Input(shape=in_shape)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output layer nodes
    fe = Dense(n_classes)(fe)
    # supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and compile supervised discriminator model
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    # unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
    # define and compile unsupervised discriminator model
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return d_model, c_model

In [4]:
# define the standalone generator model
def define_generator(latent_dim):
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model(in_lat, out_layer)
    return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect image output from generator as input to discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and outputting a classification
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
# load the images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]

In [7]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

In [8]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    z_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input

In [10]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)
    # create class labels
    y = zeros((n_samples, 1))
    return images, y

In [11]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # evaluate the classifier model
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = 'c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [12]:
# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset)

In [13]:
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(60000, 28, 28, 1) (60000,)
(100, 28, 28, 1) (100,)
n_epochs=20, n_batch=100, 1/2=50, b/e=600, steps=12000
Instructions for updating:
Use tf.cast instead.


C:\Users\Serge\Anaconda3\envs\nlp\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c[2.301,16], d[0.096,2.399], g[0.095]
>2, c[2.313,10], d[0.091,2.398], g[0.095]
>3, c[2.278,20], d[0.086,2.399], g[0.095]
>4, c[2.279,24], d[0.082,2.403], g[0.095]
>5, c[2.274,14], d[0.078,2.408], g[0.095]
>6, c[2.263,22], d[0.076,2.411], g[0.095]
>7, c[2.242,30], d[0.076,2.407], g[0.096]
>8, c[2.234,26], d[0.077,2.396], g[0.097]
>9, c[2.195,46], d[0.079,2.378], g[0.099]
>10, c[2.195,38], d[0.082,2.363], g[0.101]
>11, c[2.201,36], d[0.084,2.356], g[0.101]
>12, c[2.105,44], d[0.081,2.361], g[0.100]
>13, c[2.153,22], d[0.076,2.370], g[0.099]
>14, c[2.080,34], d[0.063,2.378], g[0.099]
>15, c[2.070,34], d[0.052,2.369], g[0.101]
>16, c[2.082,22], d[0.045,2.339], g[0.105]
>17, c[2.046,28], d[0.038,2.300], g[0.110]
>18, c[2.033,28], d[0.030,2.275], g[0.113]
>19, c[1.924,48], d[0.023,2.259], g[0.115]
>20, c[1.979,42], d[0.018,2.236], g[0.122]
>21, c[1.699,60], d[0.018,2.174], g[0.138]
>22, c[1.702,60], d[0.030,2.016], g[0.176]
>23, c[1.719,58], d[0.065,1.838], g[0.207]
>24, c[1.715,50], d[

>192, c[0.110,100], d[0.808,0.732], g[1.579]
>193, c[0.170,98], d[0.783,0.777], g[1.474]
>194, c[0.147,98], d[0.941,0.884], g[1.233]
>195, c[0.147,98], d[0.783,1.046], g[1.261]
>196, c[0.179,100], d[0.962,0.761], g[1.243]
>197, c[0.122,100], d[0.833,0.983], g[1.491]
>198, c[0.131,100], d[1.364,0.602], g[1.230]
>199, c[0.132,98], d[0.769,0.724], g[1.221]
>200, c[0.133,98], d[0.876,1.211], g[1.258]
>201, c[0.169,98], d[0.962,0.749], g[1.208]
>202, c[0.130,100], d[0.773,0.840], g[1.404]
>203, c[0.126,98], d[0.911,0.844], g[1.363]
>204, c[0.158,96], d[1.003,0.984], g[1.273]
>205, c[0.163,96], d[1.140,0.826], g[1.314]
>206, c[0.090,100], d[0.857,0.631], g[1.420]
>207, c[0.102,98], d[0.983,0.923], g[1.122]
>208, c[0.141,98], d[0.798,0.883], g[1.157]
>209, c[0.092,100], d[0.784,0.833], g[1.409]
>210, c[0.135,98], d[1.151,0.863], g[1.167]
>211, c[0.199,94], d[0.885,0.953], g[1.255]
>212, c[0.157,98], d[0.742,0.999], g[1.357]
>213, c[0.147,98], d[0.837,0.805], g[1.539]
>214, c[0.232,94], d[1.10

>379, c[0.039,100], d[0.868,1.159], g[1.108]
>380, c[0.052,100], d[1.284,1.164], g[1.176]
>381, c[0.042,100], d[0.830,1.235], g[1.073]
>382, c[0.038,100], d[0.867,1.068], g[1.127]
>383, c[0.058,98], d[1.113,1.171], g[1.220]
>384, c[0.035,100], d[1.280,0.997], g[1.028]
>385, c[0.062,100], d[1.102,1.191], g[1.066]
>386, c[0.058,100], d[0.913,0.942], g[1.306]
>387, c[0.032,100], d[1.219,1.139], g[1.298]
>388, c[0.051,100], d[0.859,0.848], g[1.377]
>389, c[0.058,100], d[0.964,0.727], g[1.345]
>390, c[0.059,100], d[1.019,0.775], g[1.758]
>391, c[0.056,100], d[0.729,0.506], g[1.989]
>392, c[0.067,100], d[0.759,0.555], g[2.018]
>393, c[0.048,100], d[0.821,0.497], g[1.926]
>394, c[0.028,100], d[0.483,0.348], g[2.278]
>395, c[0.080,96], d[0.519,0.364], g[1.966]
>396, c[0.039,100], d[0.474,0.401], g[1.813]
>397, c[0.065,100], d[0.458,0.387], g[1.747]
>398, c[0.065,100], d[0.539,0.514], g[1.758]
>399, c[0.041,100], d[0.628,0.450], g[1.954]
>400, c[0.068,98], d[0.633,0.419], g[1.829]
>401, c[0.042

...

>11295, c[0.001,100], d[1.265,0.774], g[1.001]
>11296, c[0.001,100], d[0.626,0.770], g[1.089]
>11297, c[0.001,100], d[0.684,1.097], g[1.106]
>11298, c[0.000,100], d[0.738,1.086], g[1.076]
>11299, c[0.001,100], d[0.708,1.015], g[1.113]
>11300, c[0.001,100], d[1.004,1.002], g[1.113]
>11301, c[0.001,100], d[0.713,0.610], g[1.067]
>11302, c[0.001,100], d[0.636,0.787], g[1.124]
>11303, c[0.001,100], d[0.803,0.794], g[1.096]
>11304, c[0.001,100], d[0.884,1.070], g[1.204]
>11305, c[0.001,100], d[0.695,0.929], g[1.337]
>11306, c[0.001,100], d[1.061,0.516], g[1.049]
>11307, c[0.001,100], d[0.696,0.811], g[1.254]
>11308, c[0.001,100], d[0.589,1.002], g[1.202]
>11309, c[0.000,100], d[0.806,0.851], g[1.043]
>11310, c[0.001,100], d[0.659,0.786], g[1.019]
>11311, c[0.000,100], d[0.714,0.770], g[1.227]
>11312, c[0.000,100], d[0.793,0.953], g[1.158]
>11313, c[0.001,100], d[0.924,0.748], g[1.104]
>11314, c[0.001,100], d[0.685,1.005], g[1.266]
>11315, c[0.000,100], d[0.872,0.737], g[1.193]
>11316, c[0.0

>11471, c[0.000,100], d[0.965,0.666], g[1.242]
>11472, c[0.001,100], d[0.787,1.172], g[1.059]
>11473, c[0.001,100], d[0.883,0.957], g[1.159]
>11474, c[0.000,100], d[0.748,0.884], g[1.260]
>11475, c[0.001,100], d[0.665,0.570], g[1.263]
>11476, c[0.001,100], d[0.715,0.876], g[1.092]
>11477, c[0.001,100], d[0.667,0.928], g[1.173]
>11478, c[0.001,100], d[0.852,1.036], g[1.230]
>11479, c[0.001,100], d[0.871,0.921], g[1.186]
>11480, c[0.001,100], d[0.731,0.859], g[1.066]
>11481, c[0.000,100], d[0.860,0.791], g[1.133]
>11482, c[0.001,100], d[0.821,0.757], g[1.101]
>11483, c[0.001,100], d[0.739,0.973], g[1.040]
>11484, c[0.000,100], d[0.684,0.860], g[1.137]
>11485, c[0.000,100], d[0.629,1.024], g[1.163]
>11486, c[0.000,100], d[0.630,0.711], g[1.243]
>11487, c[0.001,100], d[0.694,0.654], g[1.127]
>11488, c[0.000,100], d[0.878,1.064], g[1.263]
>11489, c[0.001,100], d[0.605,0.727], g[0.960]
>11490, c[0.001,100], d[1.002,0.993], g[1.218]
>11491, c[0.005,100], d[1.032,0.922], g[1.122]
>11492, c[0.0

>11646, c[0.001,100], d[0.697,0.860], g[1.101]
>11647, c[0.001,100], d[0.932,1.024], g[1.116]
>11648, c[0.001,100], d[0.882,0.731], g[1.044]
>11649, c[0.001,100], d[0.886,1.259], g[1.087]
>11650, c[0.001,100], d[0.900,0.787], g[0.977]
>11651, c[0.001,100], d[0.732,0.836], g[1.180]
>11652, c[0.001,100], d[0.967,0.891], g[0.952]
>11653, c[0.001,100], d[0.836,0.890], g[0.983]
>11654, c[0.001,100], d[0.842,0.780], g[1.027]
>11655, c[0.001,100], d[0.992,1.063], g[1.002]
>11656, c[0.001,100], d[0.761,0.752], g[1.058]
>11657, c[0.001,100], d[0.977,0.893], g[1.103]
>11658, c[0.001,100], d[0.699,0.779], g[0.885]
>11659, c[0.002,100], d[0.824,1.004], g[0.961]
>11660, c[0.001,100], d[0.885,0.763], g[0.987]
>11661, c[0.001,100], d[0.830,0.966], g[0.975]
>11662, c[0.001,100], d[0.752,0.733], g[0.976]
>11663, c[0.001,100], d[0.760,0.892], g[1.062]
>11664, c[0.002,100], d[0.942,0.976], g[1.015]
>11665, c[0.001,100], d[1.186,0.851], g[1.021]
>11666, c[0.001,100], d[0.735,0.849], g[1.012]
>11667, c[0.0

>11824, c[0.002,100], d[0.766,1.074], g[1.061]
>11825, c[0.001,100], d[0.829,0.760], g[1.151]
>11826, c[0.001,100], d[0.875,0.768], g[1.215]
>11827, c[0.000,100], d[0.770,0.894], g[1.058]
>11828, c[0.001,100], d[0.685,0.721], g[1.224]
>11829, c[0.001,100], d[0.699,0.649], g[1.173]
>11830, c[0.001,100], d[1.024,0.773], g[1.139]
>11831, c[0.001,100], d[0.930,0.848], g[1.052]
>11832, c[0.001,100], d[0.707,0.827], g[1.099]
>11833, c[0.001,100], d[0.764,0.878], g[1.137]
>11834, c[0.000,100], d[0.785,0.713], g[1.103]
>11835, c[0.001,100], d[0.747,0.776], g[1.195]
>11836, c[0.001,100], d[0.841,0.809], g[1.075]
>11837, c[0.001,100], d[0.825,0.810], g[1.103]
>11838, c[0.001,100], d[0.789,0.711], g[1.002]
>11839, c[0.001,100], d[0.771,0.869], g[0.974]
>11840, c[0.000,100], d[0.708,0.888], g[1.090]
>11841, c[0.001,100], d[0.704,0.905], g[1.026]
>11842, c[0.001,100], d[0.989,1.040], g[1.030]
>11843, c[0.001,100], d[0.695,0.834], g[1.146]
>11844, c[0.001,100], d[0.712,0.596], g[1.055]
>11845, c[0.0

>12000, c[0.003,100], d[0.761,0.597], g[1.004]
Classifier Accuracy: 93.968%
>Saved: generated_plot_12000.png, g_model_12000.h5, and c_model_12000.h5
